#### Open Image Dataset의 Object Detection 학습 및 Inference
* Open Image Dataset에서 Football 관련 Object, Fish관련 Object를 추출 후 학습 데이터 세트 생성. 
* 이를 이용하여 Object Detection 수행. 

### 본 실습 예제는 GPU를 활용하므로 상단 메뉴에서 런타임->런타임 유형 변경에서 GPU를 선택해 주십시요.

### tensorflow, keras 설치 및 강의 실습코드/데이터 Download

#### 공지

현재(2020년 8월 11일) Colab에서 GPU 커널 적용시 tensorflow 1.13으로 downgrade가 되지 않습니다. 때문에 colab에서는 Segmentation 학습 시 tensorflow 1.15, keras 2.3 을 설치하겠습니다.

Colab 버전colab에서 pip 명령어를 이용하여 tensorflow 1.15, keras 2.3를 소스코드 커널 기동시 마다 설치해야 합니다

Colab의 tensorflow는 2020년 기준으로 2.2 이며, keras는 2.3입니다. 실습코드는 tensorflow 1.13과 1.15, keras 2.2와 2.3 기준으로 되어 있으므로 이를 downgrade해야 합니다.

pip를 이용하여 tensorflow 1.15을 설치하면 자동으로 downgrade 됩니다.

OpenCV는 Colab에서 이미 설치 되어 있으니 추가설치는 필요 없습니다.

강의 실습코드와 데이터는 https://github.com/chulminkw/DLCV.git 에서 다운로드 할 수 있습니다. 

In [ ]:
# 현재 디렉토리는 /content이며 이 디렉토리를 기준으로 실습코드와 데이터를 다운로드 합니다. 
!pwd
!rm -rf DLCV
!git clone https://github.com/chulminkw/DLCV.git
# DLCV 디렉토리가 Download되고 DLCV 밑에 Detection과 Segmentation 디렉토리가 있는 것을 확인
!ls -lia 
!ls -lia DLCV

#현재(2020년 8월 11일) Colab에서 GPU 커널 적용시 tensorflow 1.13으로 downgrade가 되지 않습니다. 
#때문에 colab에서는 Segmentation 학습 시 tensorflow 1.15, keras 2.3 을 설치하겠습니다.
# tensorflow 1.15을 설치합니다. 자동으로 tensorflow 2.2가 1.15로 downgrade 됩니다. 
!pip install tensorflow-gpu==1.15.2 
# keras 2.3를 설치합니다. 
!pip install keras==2.3.0

### 중요. 반드시 아래 import tensorflow, import keras 수행 전 이 셀을 먼저 수행해야 합니다. 
#### Keras-yolo3 는 Custom data 를 train시 오류가 발생하는등 tensorflow 1.15 와 완벽하게 호환하지 않습니다. 
#### 때문에 아래와 같이 DLCV github에서 수정된 __init__.py 를 다운로드 받아서 keras의 backend의 __init__.py 를 수정해야 합니다.
#### __init__.py는 반드시 import tensorflow, import keras 이전에 수행되어야 합니다. 만일 tensorflow, keras 설치한 뒤 아래의 import tensorflow, import keras를 먼저 수행하였으면 메뉴-> 런타임 -> 런타임 다시 시작을 누르신 뒤 __init__.py를 수정합니다. 

In [ ]:
import os

# keras backend 디렉토리 이동. 
os.chdir('/usr/local/lib/python3.6/dist-packages/keras/backend')

!rm -rf __init__.py
!rm -rf __pycache__
# 기존 __init__.py 삭제하고 새로운 __init__.py를 download 
!wget https://raw.githubusercontent.com/chulminkw/DLCV/master/colab_tf115_modify_files/__init__.py

In [ ]:
# tensorflow는 1.15, keras는 2.3 버전 확인
# GPU가 세팅되어 있지 않으면 상단 메뉴에서 런타임->런타임 유형 변경에서 GPU를 선택한 후 런타임 다시 시작을 선택하고 처음 부터인 tensorflow, keras 설치 부터 다시 시작. 
import tensorflow as tf
import keras

print(tf.__version__)
print(keras.__version__)

# gpu가 세팅드어 있는지 확인. 
tf.test.gpu_device_name()

### 코랩 버전은 아래를 이용하여 keras-yolo3 패키지를 download하여 /content/DLCV/Detection/yolo 밑에 설치 

In [ ]:
%cd /content/DLCV/Detection/yolo
!git clone https://github.com/qqwweee/keras-yolo3.git
!ls -lia /content/DLCV/Detection/yolo/keras-yolo3


### 코랩 버전 pretrained 모델 재생성 및 font 디렉토리 교체  
* 코랩 버전은 다시 model_data 밑에 coco dataset로 pretrained 된 yolov3.weights 파일을 yolo.h5 파일로 변경해 줘야 합니다. 
* keras-yolo3의 font 디렉토리도 재 교체 합니다. 

In [ ]:
# 코랩 버전은 아래를 이용하여 yolov3.weights 파일을 download 받고, convert.py 를 수행하여 model_data 밑에 yolo.h5 파일 생성 수행. 
%cd /content/DLCV/Detection/yolo/keras-yolo3 
# yolo 공식 사이트에서 download시 download 속도가 약 25분 정도 소요됨. 강의 github에서 다운로드 요망. 
#!wget  https://pjreddie.com/media/files/yolov3.weights
# 강의 실습 github에서 다운로드
!wget https://github.com/chulminkw/DLCV/releases/download/1.0/yolov3.weights

# yolov3.weights를 keras-yolo3에서 사용할 수 있도록 yolo.h5 로 변환
!python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5
# model_data 밑에 yolo.h5 파일이 생성되었는지 확인. 
!ls /content/DLCV/Detection/yolo/keras-yolo3/model_data
# yolo.detect_image() 메소드는 PIL package를 이용하여 image 작업 수행. keras-yolo3/font 디렉토리를 상위 디렉토리로 복사 해야함.  
%cd /content/DLCV/Detection/yolo
!cp -rf keras-yolo3/font ./font

#### 코랩 버전은 이미 OID Toolkit을 통해 만들어진 데이터 세트를 다운로드 합니다. 
* 코랩 버전은 OID Toolkit을 사용하는 동영상 강의를 참조하지 않으셔도 됩니다. 
* 이미 만들어진 ballnfish.tar 파일을 다운로드 한 뒤 이를 이용하여 학습 합니다. 

In [ ]:
# ballnfish.tar 파일을 강의의 github에서 download 한 뒤 압축을 풀면
# /content/DLCV/data/ballnfish 디렉토리로 annotations, images 디렉토리와 관련 파일들이 생성 됩니다. 
%cd /content/DLCV/data
!wget https://github.com/chulminkw/DLCV/releases/download/1.0/ballnfish.tar
!tar -xvf ballnfish.tar > /dev/null 2>&1

In [ ]:
# annotation과 image 디렉토리 설정. annotation디렉토리에 있는 파일 확인. 
import os
from pathlib import Path

#HOME_DIR = str(Path.home())
# 코랩 버전은 HOME_DIR을 /content 로 설정합니다.
HOME_DIR = '/content'

ANNO_DIR = os.path.join(HOME_DIR, 'DLCV/data/ballnfish/annotations')
IMAGE_DIR = os.path.join(HOME_DIR, 'DLCV/data/ballnfish/images')
print(ANNO_DIR)

files = os.listdir(ANNO_DIR)
print('파일 개수는:',len(files))
print(files)

In [ ]:
# 아래는 위에서 출력된 적당한 xml 파일로 변경되어야 합니다. 
!cat /content/DLCV/data/ballnfish/annotations/13825baad5531265.xml

In [ ]:
import glob
import xml.etree.ElementTree as ET

classes_map = {'Football':0, 'Football_helmet':1, 'Fish':2, 'Shark':3, 'Shellfish':4 }

def xml_to_csv(path, output_filename):
    xml_list = []
    # xml 확장자를 가진 모든 파일의 절대 경로로 xml_file할당. 
    with open(output_filename, "w") as train_csv_file:
        for xml_file in glob.glob(path + '/*.xml'):
            # xml 파일을 parsing하여 XML Element형태의 Element Tree를 생성하여 object 정보를 추출. 
            tree = ET.parse(xml_file)
            root = tree.getroot()
            print('xml file:', xml_file)
            # 파일내에 있는 모든 object Element를 찾음. 
            full_image_name = os.path.join(IMAGE_DIR, root.find('filename').text)
            value_str_list = ' '
            for obj in root.findall('object'):
                xmlbox = obj.find('bndbox')
                class_name = obj.find('name').text
                x1 = int(xmlbox.find('xmin').text)
                y1 = int(xmlbox.find('ymin').text)
                x2 = int(xmlbox.find('xmax').text)
                y2 = int(xmlbox.find('ymax').text)
                # 
                class_id = classes_map[class_name]
                value_str = ('{0},{1},{2},{3},{4}').format(x1, y1, x2, y2, class_id)
                # object별 정보를 tuple형태로 object_list에 저장. 
                value_str_list = value_str_list+value_str+' '
        
            train_csv_file.write(full_image_name+' '+ value_str_list+'\n')
        # xml file 찾는 for loop 종료 

In [ ]:
xml_to_csv(ANNO_DIR, os.path.join(ANNO_DIR,'ballnfish_anno.csv'))
print(os.path.join(ANNO_DIR,'ballnfish_anno.csv'))

In [ ]:
!cat /content/DLCV/data/ballnfish/annotations/ballnfish_anno.csv

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

default_dir = '/content'
# 9c27811a78b74a48.jpg 는 images 디렉토리에 있는 임의 파일로 바뀌어야 합니다. 
plt.imshow(cv2.cvtColor(cv2.imread(os.path.join(default_dir, 'DLCV/data/ballnfish/images/9c27811a78b74a48.jpg')), cv2.COLOR_BGR2RGB))

In [ ]:
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
import sys, os

import os

# 코랩 버전은 아래와 같이 절대 경로를 지정하여 Local Package 지정. 
default_dir = '/content/DLCV'
default_yolo_dir = os.path.join(default_dir, 'Detection/yolo')

LOCAL_PACKAGE_DIR = os.path.abspath(os.path.join(default_yolo_dir,'keras-yolo3'))
print(LOCAL_PACKAGE_DIR)
sys.path.append(LOCAL_PACKAGE_DIR)

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

#### colab 버전은 아래 명령어로 ballnfish_classes.txt 를 수정

In [ ]:
# colab 버전은 아래 명령어로 ballnfish_classes.txt 를 수정합니다. 
BASE_DIR = os.path.join(HOME_DIR, 'DLCV/Detection/yolo/keras-yolo3')
classes_path = os.path.join(BASE_DIR, 'model_data/ballnfish_classes.txt')
with open(classes_path, "w") as f:
    f.write("Football\n")
    f.write("Football_Helmet\n")
    f.write("Fish\n")
    f.write("Shark\n")
    f.write("Shell_Fish\n")

# colab 버전은 raccoon_class.txt에 제대로 기재되었나 확인. 
!cat /content/DLCV/Detection/yolo/keras-yolo3/model_data/ballnfish_classes.txt

In [ ]:
from train import get_classes, get_anchors
from train import create_model, data_generator, data_generator_wrapper

BASE_DIR = os.path.join(HOME_DIR, 'DLCV/Detection/yolo/keras-yolo3')

## 학습을 위한 기반 환경 설정. annotation 파일 위치, epochs시 저장된 모델 파일, Object클래스 파일, anchor 파일.
annotation_path = os.path.join(ANNO_DIR, 'ballnfish_anno.csv')
log_dir = os.path.join(BASE_DIR, 'snapshots/ballnfish/')
classes_path = os.path.join(BASE_DIR, 'model_data/ballnfish_classes.txt')
anchors_path = os.path.join(BASE_DIR,'model_data/yolo_anchors.txt')

class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)
print(class_names, num_classes)
print(anchors)

#### yolo 모델 학습을 위한 전반적인 파라미터를 config 클래스로 설정하고 필요시 이를 수정하여 학습. 

In [ ]:
# csv annotation 파일을 읽어서 lines 리스트로 만듬. 
with open(annotation_path) as f:
    lines = f.readlines()

class config:
    #tiny yolo로 모델로 초기 weight 학습 원할 시 아래를 tiny-yolo.h5로 수정. 
    initial_weights_path=os.path.join(BASE_DIR, 'model_data/yolo.h5' )
    # input_shape는 고정. 
    input_shape=(416, 416)
    # epochs는 freeze, unfreeze 2 step에 따라 설정. 
    first_epochs=50
    first_initial_epochs=0
    second_epochs=100
    second_initial_epochs=50
    # 학습시 batch size, train,valid건수, epoch steps 횟수  
    batch_size = 4
    val_split = 0.1   
    num_val = int(len(lines)*val_split)
    num_train = len(lines) - num_val
    train_epoch_steps = num_train//batch_size 
    val_epoch_steps =  num_val//batch_size
    
    anchors = get_anchors(anchors_path)
    class_names = get_classes(classes_path)
    num_classes = len(class_names)
    # epoch시 저장된 weight 파일 디렉토리 
    log_dir = os.path.join(BASE_DIR, 'snapshots/ballnfish/')
    
print('Class name:', config.class_names,'\nNum classes:', config.num_classes)

#### csv 파일을 입력 받아서 train 데이터와 valid 데이터 처리를 위한 data_generator_wrapper객체를 각각 생성.
* train용, valid 용 data_generator_wrapper는 Yolo 모델의 fit_generator()학습시 인자로 입력됨. 

In [ ]:
def create_generator(lines):
    
    train_data_generator = data_generator_wrapper(lines[:config.num_train], config.batch_size, 
                                                  config.input_shape, config.anchors, config.num_classes)
    
    valid_data_generator = data_generator_wrapper(lines[config.num_train:], config.batch_size, 
                                                  config.input_shape, config.anchors, config.num_classes)
    
    return train_data_generator, valid_data_generator

#### YOLO 모델 또는 tiny yolo 모델 반환. 초기 weight값은 pretrained된 yolo weight값으로 할당. 

In [ ]:
# anchor 개수에 따라 tiny yolo 모델 또는 yolo 모델 반환. 
def create_yolo_model():
    is_tiny_version = len(config.anchors)==6 
    if is_tiny_version:
        model = create_tiny_model(config.input_shape, config.anchors, config.num_classes, 
            freeze_body=2, weights_path=config.initial_weights_path)
    else:
        model = create_model(config.input_shape, config.anchors, config.num_classes, 
            freeze_body=2, weights_path=config.initial_weights_path)
        
    return model 

#### callback 객체들을 생성. 

In [ ]:
# Tensorboard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping callback 반환
def create_callbacks():
    logging = TensorBoard(log_dir=config.log_dir)
    checkpoint = ModelCheckpoint(config.log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
        monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)
    
    #개별 callback들을 한꺼번에 list로 묶어서 반환
    return [logging, checkpoint, reduce_lr, early_stopping]

#### 학습 수행

In [ ]:
   
# create_generator(), create_model(), create_callbacks() 수행. 
train_data_generator, valid_data_generator = create_generator(lines)
ballnfish_model = create_yolo_model()
callbacks_list = create_callbacks()

# 최초 모델은 주요 layer가 freeze되어 있음. 안정적인 loss를 확보하기 위해 주요 layer를 freeze한 상태로 먼저 학습. 
print('First train 시작' )
ballnfish_model.compile(optimizer=Adam(lr=1e-3), loss={'yolo_loss': lambda y_true, y_pred: y_pred})

ballnfish_model.fit_generator(train_data_generator, steps_per_epoch=config.train_epoch_steps,
        validation_data=valid_data_generator, validation_steps=config.val_epoch_steps,
        epochs=config.first_epochs, initial_epoch=config.first_initial_epochs, 
        callbacks=callbacks_list)

# 1단계 학습 완료 모델 저장. 
ballnfish_model.save_weights(log_dir + 'trained_weights_stage_1.h5')

# 모든 layer를 trainable=True로 설정하고 학습 수행. 
for i in range(len(ballnfish_model.layers)):
    ballnfish_model.layers[i].trainable = True
    
print('Second train 시작' )
ballnfish_model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) 
ballnfish_model.fit_generator(train_data_generator, steps_per_epoch=config.train_epoch_steps,
    validation_data=valid_data_generator, validation_steps=config.val_epoch_steps,
    epochs=config.second_epochs, initial_epoch=config.second_initial_epochs,
    callbacks=callbacks_list)

# 최종 학습 완료 모델 저장. 
ballnfish_model.save_weights(log_dir + 'trained_weights_final.h5')

#### 최종 학습된 모델을 로딩하여 Object Detection 수행. 

In [ ]:
from yolo import YOLO
#keras-yolo에서 image처리를 주요 PIL로 수행. 
from PIL import Image

# 코랩 버전은 아래와 같이 절대 경로를 지정하여 Local Package 지정. 
default_dir = '/content/DLCV'
default_yolo_dir = os.path.join(default_dir, 'Detection/yolo')

LOCAL_PACKAGE_DIR = os.path.abspath(os.path.join(default_yolo_dir,'keras-yolo3'))
sys.path.append(LOCAL_PACKAGE_DIR)

ballnfish_yolo = YOLO(model_path='/content/DLCV/Detection/yolo/keras-yolo3/snapshots/ballnfish/trained_weights_final.h5',
            anchors_path='/content/DLCV/Detection/yolo/keras-yolo3/model_data/yolo_anchors.txt',
            classes_path='/content/DLCV/Detection/yolo/keras-yolo3/model_data/ballnfish_classes.txt')

#### 이미지 Object Detection

In [ ]:
%cd /content/DLCV/data/ballnfish
!ls annotations

In [ ]:
# 아래 football_list는 적절한 jpg 파일명으로 변경되어야 합니다. 
football_list = ['f1b492a9bce3ac9a.jpg', '1e6ff631bb0c198b.jpg', '97ac013310bda756.jpg',
                'e5b1646c395aecfd.jpg', '53ef241dad498f6c.jpg', '02ccbf5ddaaecedb.jpg' ]
for image_name in football_list:
    img = Image.open(os.path.join(IMAGE_DIR, image_name))
    detected_img = ballnfish_yolo.detect_image(img)
    plt.figure(figsize=(8, 8))
    plt.imshow(detected_img)
    

In [ ]:
# 아래 helmet_list는 적절한 jpg 파일명으로 변경되어야 합니다. 
helmet_list = ['1fed5c930211c6e0.jpg', '011a59a160d7a091.jpg', 'd39b46aa4bc0c165.jpg', '7e9eb7eba80e34e7.jpg', '9c27811a78b74a48.jpg']
for image_name in helmet_list:
    img = Image.open(os.path.join(IMAGE_DIR, image_name))
    detected_img = ballnfish_yolo.detect_image(img)
    plt.figure(figsize=(8, 8))
    plt.imshow(detected_img)

In [ ]:
# 아래 fish_list는 적절한 jpg 파일명으로 변경되어야 합니다.
fish_list = ['25e42c55bfcbaa88.jpg', 'a571e4cdcfbcb79e.jpg', '872c435491f2b4d3.jpg', 
             'bebac23c45451d93.jpg', 'eba7caf07a26829b.jpg', 'dc607a2989bdc9dc.jpg' ]
for image_name in fish_list:
    img = Image.open(os.path.join(IMAGE_DIR, image_name))
    detected_img = ballnfish_yolo.detect_image(img)
    plt.figure(figsize=(8, 8))
    plt.imshow(detected_img)

In [ ]:
#아래 shark_list는 적절한 jpg 파일명으로 변경되어야 합니다.
shark_list = ['d92290f6c04dd83b.jpg', '3a37a09ec201cdeb.jpg', '32717894b5ce0052.jpg', 'a848df5dbed78a0f.jpg', '3283eafe11a847c3.jpg']
for image_name in shark_list:
    img = Image.open(os.path.join(IMAGE_DIR, image_name))
    detected_img = ballnfish_yolo.detect_image(img)
    plt.figure(figsize=(8, 8))
    plt.imshow(detected_img)

In [ ]:
#아래 shell_list는 적절한 jpg 파일명으로 변경되어야 합니다.
shell_list=['5cc89bc28084e8e8.jpg',  '055e756883766e1f.jpg', '089354fc39f5d82d.jpg', '80eddfdcb3384458.jpg']
for image_name in shell_list:
    img = Image.open(os.path.join(IMAGE_DIR, image_name))
    detected_img = ballnfish_yolo.detect_image(img)
    plt.figure(figsize=(8, 8))
    plt.imshow(detected_img)

#### keras-yolo 패키지의 font 디렉토리 이동 

In [ ]:
%cd /content/DLCV/Detection/yolo
!cp -rf keras-yolo3/font ./font

#### 영상 Object Detection 

In [ ]:
import cv2
import time

def detect_video_yolo(model, input_path, output_path=""):
    
    start = time.time()
    cap = cv2.VideoCapture(input_path)
    
    #codec = cv2.VideoWriter_fourcc(*'DIVX')
    codec = cv2.VideoWriter_fourcc(*'XVID')
    vid_fps = cap.get(cv2.CAP_PROP_FPS)
    vid_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    vid_writer = cv2.VideoWriter(output_path, codec, vid_fps, vid_size)
    
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('총 Frame 갯수:', frame_cnt, '원본 영상 FPS:',vid_fps, '원본 Frame 크기:', vid_size)
    index = 0
    while True:
        hasFrame, image_frame = cap.read()
        if not hasFrame:
            print('프레임이 없거나 종료 되었습니다.')
            break
        start = time.time()
        # PIL Package를 내부에서 사용하므로 cv2에서 읽은 image_frame array를 다시 PIL의 Image형태로 변환해야 함.  
        image = Image.fromarray(image_frame)
        # 아래는 인자로 입력된 yolo객체의 detect_image()로 변환한다.
        detected_image = model.detect_image(image)
        # cv2의 video writer로 출력하기 위해 다시 PIL의 Image형태를 array형태로 변환 
        result = np.asarray(detected_image)
        index +=1
        print('#### frame:{0} 이미지 처리시간:{1}'.format(index, round(time.time()-start,3)))
        
        vid_writer.write(result)
    
    vid_writer.release()
    cap.release()
    print('### Video Detect 총 수행시간:', round(time.time()-start, 5))

In [ ]:
detect_video_yolo(ballnfish_yolo, '/content/DLCV/data/video/NFL01.mp4', '/content/DLCV/data/output/NFL_yolo_01.avi')

In [ ]:
## colab 버전은 Object Detection 적용된 영상 파일을 google drive에서 download 해야 합니다. 이를 위해 google drive를 colab에 mount 수행. 
import os, sys 
from google.colab import drive 

drive.mount('/content/gdrive')

In [ ]:
## colab 버전은 Object Detection 적용된 영상 파일을 google drive에서 download 해야 합니다. 
## My Drive 디렉토리 이름에 공란이 있으므로 ' '로 묶습니다. 
!cp /content/DLCV/data/output/NFL_yolo_01.avi '/content/gdrive/My Drive/NFL_yolo_01.avi'

In [ ]:
detect_video_yolo(ballnfish_yolo, '/content/DLCV/data/video/FishnShark01.mp4', '/content/DLCV/data/output/FishnShark_yolo_01.avi')

In [ ]:
## colab 버전은 Object Detection 적용된 영상 파일을 google drive에서 download 해야 합니다. 
## My Drive 디렉토리 이름에 공란이 있으므로 ' '로 묶습니다. 
!cp /content/DLCV/data/output/FishnShark_yolo_01.avi '/content/gdrive/My Drive/FishnShark_yolo_01.avi'